<a href="https://colab.research.google.com/github/mamekin05108/signatecup2024summer/blob/main/Catboost_20240825.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
import copy
import lightgbm as lgb
from sklearn import metrics
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold

# warningsを非表示にする
warnings.filterwarnings("ignore")

In [5]:

!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 7.7 MB/s eta 0:00:00


In [50]:
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)

In [6]:

from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier, Pool

In [57]:
df_train = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/fdata/df_train5.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/fdata/df_test5.csv")
ss = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/data/sample_submit.csv", header=None)

In [96]:
df_train.head(18)

,id,Age,TypeofContact,DurationOfPitch,Gender,NumberOfPersonVisiting,NumberOfFollowups,NumberOfTrips,Passport,MonthlyIncome,ProdTaken,OwnCar,NumberOfChildren,HasChildren,CityTier_1,CityTier_2,CityTier_3,Occupation_0,Occupation_1,Occupation_2,ProductPitched_BASIC,ProductPitched_DELUXE,ProductPitched_KING,ProductPitched_STANDARD,ProductPitched_SUPERDELUXE,PreferredPropertyStar_3.0,PreferredPropertyStar_4.0,PreferredPropertyStar_5.0,PitchSatisfactionScore_1,PitchSatisfactionScore_2,PitchSatisfactionScore_3,PitchSatisfactionScore_4,PitchSatisfactionScore_5,Designation_AVP,Designation_Executive,Designation_Manager,Designation_Senior Manager,Designation_VP,MaritalStatus_未婚,MaritalStatus_独身,MaritalStatus_結婚,MaritalStatus_離婚,IncomePerAge,FamilySize,IncomePerFamilySize,IncomePerTrips,PersonTripsProduct,IncomePerChild
0,0,50.00000,1,15.0,0,1.0,4.0,5.0,1,253,1.0,0,0.0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,5.060000,2.0,126.5,50.600000,5.0,inf
1,1,56.00000,0,14.0,0,1.0,4.0,2.0,1,404,0.0,1,0.0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,7.214286,1.0,404.0,202.000000,2.0,inf
2,2,41.13181,1,10.0,1,1.0,3.0,4.0,0,278,1.0,0,0.0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,6.758759,2.0,139.0,69.500000,4.0,inf
3,3,37.00000,1,18.0,1,1.0,3.0,1.0,0,326,0.0,1,0.0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,8.810811,1.0,326.0,326.000000,1.0,inf
4,4,48.00000,0,17.0,1,1.0,3.0,4.0,0,258,1.0,1,0.0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,5.375000,1.0,258.0,64.500000,4.0,inf
5,5,19.00000,1,16.0,0,1.0,3.0,2.0,0,260,0.0,0,0.0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,13.684211,1.0,260.0,130.000000,2.0,inf
6,6,47.00000,0,14.0,0,1.0,3.0,1.0,0,453,0.0,1,0.0,0,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,9.638298,1.0,453.0,453.000000,1.0,inf
7,7,49.00000,1,22.0,0,1.0,3.0,2.0,0,261,0.0,0,0.0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,5.326531,2.0,130.5,130.500000,2.0,inf
8,8,33.00000,0,20.0,1,1.0,3.0,4.0,0,247,1.0,0,0.0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,7.484848,1.0,247.0,61.750000,4.0,inf
9,9,31.00000,0,11.0,0,1.0,3.0,2.0,0,257,0.0,1,0.0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,8.290323,2.0,128.5,128.500000,2.0,inf


In [59]:
# Config cell
target = "ProdTaken"

In [118]:

train_y = df_train[target]
train_x = df_train.drop(target, axis=1)

In [119]:
default_categorical_features = ['TypeofContact', 'CityTier', 'Occupation', 'Gender', 'ProductPitched', 'PreferredPropertyStar',
                                'Passport', 'PitchSatisfactionScore', 'Designation', 'MaritalStatus']
default_numerical_features = ['id','Age', 'DurationOfPitch', 'MonthlyIncome',
                            'NumberOfPersonVisiting', 'NumberOfFollowups', 'NumberOfTrips', 'NumberOfChildren','OwnCar'
                              ]
added_numerical_features = ['IncomePerAge','FamilySize','IncomePerFamilySize','IncomePerTrips','PersonTripsProduct','IncomePerChild']
numerical_features = default_numerical_features + added_numerical_features



#特徴量の指定
features = df_train.columns.tolist()
#カテゴリカル特徴量の指定
categorical_features = copy.deepcopy(features)
print(f'numerical_features {numerical_features}')
for i in numerical_features:
    categorical_features.remove(i)
print(f'categorical_features {categorical_features}')

#学習に使用しない特徴量は以下で除外
RemoveList=[target, 'id']
for i in RemoveList:
    if i in numerical_features:
        numerical_features.remove(i)
        features.remove(i)
    elif i in categorical_features:
        categorical_features.remove(i)
        features.remove(i)
    else:
        features.remove(i)
print(f'features for training:{features}')

for col in categorical_features:
    df_train[col] = df_train[col].astype(int)
    df_test[col] = df_test[col].astype(int)

numerical_features ['id', 'Age', 'DurationOfPitch', 'MonthlyIncome', 'NumberOfPersonVisiting', 'NumberOfFollowups', 'NumberOfTrips', 'NumberOfChildren', 'OwnCar', 'IncomePerAge', 'FamilySize', 'IncomePerFamilySize', 'IncomePerTrips', 'PersonTripsProduct', 'IncomePerChild']
categorical_features ['TypeofContact', 'Gender', 'Passport', 'ProdTaken', 'HasChildren', 'CityTier_1', 'CityTier_2', 'CityTier_3', 'Occupation_0', 'Occupation_1', 'Occupation_2', 'ProductPitched_BASIC', 'ProductPitched_DELUXE', 'ProductPitched_KING', 'ProductPitched_STANDARD', 'ProductPitched_SUPERDELUXE', 'PreferredPropertyStar_3.0', 'PreferredPropertyStar_4.0', 'PreferredPropertyStar_5.0', 'PitchSatisfactionScore_1', 'PitchSatisfactionScore_2', 'PitchSatisfactionScore_3', 'PitchSatisfactionScore_4', 'PitchSatisfactionScore_5', 'Designation_AVP', 'Designation_Executive', 'Designation_Manager', 'Designation_Senior Manager', 'Designation_VP', 'MaritalStatus_未婚', 'MaritalStatus_独身', 'MaritalStatus_結婚', 'MaritalStatus_離

In [120]:
cols_category=categorical_features

In [135]:

params_cat = {
    "iterations": 1000,  # n_estimatorsに相当
    'eval_metric' : 'AUC',
    'depth':1,
    'l2_leaf_reg' : 8,
    "learning_rate": 0.1,
    "colsample_bylevel": 0.8,  # colsample_bytreeに相当
    "random_seed": 0,
    "verbose": 100,  # 学習の進捗を100回毎に表示
    "use_best_model": True,  # 早期停止のための設定
}

In [122]:

# f1スコアが最も高くなる点を見つける
from sklearn import metrics
def decide_cutoff(val_y, preds_y_proba):
    mean_f1_list = []
    fpr, tpr, thresholds = metrics.roc_curve(val_y, preds_y_proba)
    for threshold in thresholds:
        preds_y = [1 if prob > threshold else 0 for prob in preds_y_proba]
        #preds_y=preds_y_proba
        mean_f1_list.append(f1_score(val_y, preds_y, average='macro'))
    return np.max(mean_f1_list), thresholds[np.argmax(mean_f1_list)]

In [123]:
cols_category

['TypeofContact',
 'Gender',
 'Passport',
 'HasChildren',
 'CityTier_1',
 'CityTier_2',
 'CityTier_3',
 'Occupation_0',
 'Occupation_1',
 'Occupation_2',
 'ProductPitched_BASIC',
 'ProductPitched_DELUXE',
 'ProductPitched_KING',
 'ProductPitched_STANDARD',
 'ProductPitched_SUPERDELUXE',
 'PreferredPropertyStar_3.0',
 'PreferredPropertyStar_4.0',
 'PreferredPropertyStar_5.0',
 'PitchSatisfactionScore_1',
 'PitchSatisfactionScore_2',
 'PitchSatisfactionScore_3',
 'PitchSatisfactionScore_4',
 'PitchSatisfactionScore_5',
 'Designation_AVP',
 'Designation_Executive',
 'Designation_Manager',
 'Designation_Senior Manager',
 'Designation_VP',
 'MaritalStatus_未婚',
 'MaritalStatus_独身',
 'MaritalStatus_結婚',
 'MaritalStatus_離婚']

In [124]:
feature_name = cols_category[13]
print(f"Feature at index 13: {feature_name}")
print(df_train[feature_name].dtype)

Feature at index 13: ProductPitched_STANDARD
int64


In [125]:
for col in cols_category:
    print(f"{col}: {df_train[col].dtype}")

TypeofContact: int64
Gender: int64
Passport: int64
HasChildren: int64
CityTier_1: int64
CityTier_2: int64
CityTier_3: int64
Occupation_0: int64
Occupation_1: int64
Occupation_2: int64
ProductPitched_BASIC: int64
ProductPitched_DELUXE: int64
ProductPitched_KING: int64
ProductPitched_STANDARD: int64
ProductPitched_SUPERDELUXE: int64
PreferredPropertyStar_3.0: int64
PreferredPropertyStar_4.0: int64
PreferredPropertyStar_5.0: int64
PitchSatisfactionScore_1: int64
PitchSatisfactionScore_2: int64
PitchSatisfactionScore_3: int64
PitchSatisfactionScore_4: int64
PitchSatisfactionScore_5: int64
Designation_AVP: int64
Designation_Executive: int64
Designation_Manager: int64
Designation_Senior Manager: int64
Designation_VP: int64
MaritalStatus_未婚: int64
MaritalStatus_独身: int64
MaritalStatus_結婚: int64
MaritalStatus_離婚: int64


In [126]:
for col in cols_category:
    unique_values = df_train[col].unique()
    print(f"Unique values in {col}: {unique_values}")

Unique values in TypeofContact: [1 0]
Unique values in Gender: [0 1]
Unique values in Passport: [1 0]
Unique values in HasChildren: [0 1]
Unique values in CityTier_1: [0 1]
Unique values in CityTier_2: [1 0]
Unique values in CityTier_3: [0 1]
Unique values in Occupation_0: [0 1]
Unique values in Occupation_1: [0 1]
Unique values in Occupation_2: [1 0]
Unique values in ProductPitched_BASIC: [1 0]
Unique values in ProductPitched_DELUXE: [0 1]
Unique values in ProductPitched_KING: [0 1]
Unique values in ProductPitched_STANDARD: [0 1]
Unique values in ProductPitched_SUPERDELUXE: [0 1]
Unique values in PreferredPropertyStar_3.0: [1 0]
Unique values in PreferredPropertyStar_4.0: [0 1]
Unique values in PreferredPropertyStar_5.0: [0 1]
Unique values in PitchSatisfactionScore_1: [0 1]
Unique values in PitchSatisfactionScore_2: [0 1]
Unique values in PitchSatisfactionScore_3: [0 1]
Unique values in PitchSatisfactionScore_4: [1 0]
Unique values in PitchSatisfactionScore_5: [0 1]
Unique values in 

In [127]:
df_train['ProductPitched_STANDARD'].unique()


array([0, 1])

In [138]:
list_metrics_auc = []
list_metrics_f1 = []
list_cutoff = []
list_models = []
fraction_of_positives_list = []
mean_predicted_value_list = []

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
for fold, (trn_idx, val_idx) in enumerate(cv.split(train_x, train_y), start=1):
    trn_x = train_x.iloc[trn_idx, :]
    trn_y = train_y[trn_idx]
    val_x = train_x.iloc[val_idx, :]
    val_y = train_y[val_idx]
    train_pool = Pool(data=trn_x, label=trn_y, cat_features=cols_category)
    val_pool = Pool(data=val_x, label=val_y, cat_features=cols_category)

    model_cat = CatBoostClassifier(**params_cat, early_stopping_rounds=100)
    model_cat.fit(train_pool, eval_set=val_pool)

    list_models.append(model_cat)
    preds_y_proba = model_cat.predict_proba(val_pool)[:, 1]
    auc = roc_auc_score(val_y, preds_y_proba)
    f1, threshold = decide_cutoff(val_y, preds_y_proba)
    list_metrics_auc.append(auc)
    list_metrics_f1.append(f1)
    list_cutoff.append(threshold)
    print(f"Fold: {fold}, AUC: {auc}, f1 score: {f1} Threshold: {threshold}")

        # キャリブレーション曲線を計算
    fraction_of_positives, mean_predicted_value = calibration_curve(val_y, preds_y_proba, n_bins=10)
    fraction_of_positives_list.append(fraction_of_positives)
    mean_predicted_value_list.append(mean_predicted_value)

  # AUC
print(np.mean(list_metrics_auc), np.mean(list_metrics_f1), np.median(list_cutoff))

0:	test: 0.6007692	best: 0.6007692 (0)	total: 3.02ms	remaining: 3.02s
100:	test: 0.8671405	best: 0.8671405 (100)	total: 192ms	remaining: 1.71s
200:	test: 0.8718896	best: 0.8718896 (200)	total: 308ms	remaining: 1.23s
300:	test: 0.8694816	best: 0.8719900 (201)	total: 424ms	remaining: 985ms
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.8719899666
bestIteration = 201

Shrink model to first 202 iterations.
Fold: 1, AUC: 0.8719899665551839, f1 score: 0.7766995467875233 Threshold: 0.32076427676079455
0:	test: 0.6009699	best: 0.6009699 (0)	total: 1.28ms	remaining: 1.28s
100:	test: 0.8306940	best: 0.8322742 (92)	total: 119ms	remaining: 1.05s
200:	test: 0.8306187	best: 0.8328177 (145)	total: 242ms	remaining: 961ms
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.8328177258
bestIteration = 145

Shrink model to first 146 iterations.
Fold: 2, AUC: 0.8328177257525083, f1 score: 0.7279632485861651 Threshold: 0.29149062027219175
0:	test: 0.5195359	best: 0.519

In [139]:
import pickle

In [140]:
# モデルを保存する
# 全てのモデルを保存する
for fold, model_cat in enumerate(list_models, start=1):
    with open(f'/content/drive/MyDrive/Signate/2024summer/models/catboost_fold_v2{fold}.pkl', 'wb') as f:
        pickle.dump(model_cat, f)

In [141]:


threshold = np.median(list_cutoff)
preds_y_proba = np.zeros(len(df_test))
for model in list_models:
    feature_names = model.feature_names_
    preds_y_proba += model.predict_proba(df_test[feature_names])[:, 1] / len(list_models)
    print(f"Model: {model}, Proba: {preds_y_proba}")  # 予測確率を表示
preds_y = preds_y_proba



Model: <catboost.core.CatBoostClassifier object at 0x79214d633c10>, Proba: [0.01711891 0.03601365 0.0483567  ... 0.1196983  0.04468277 0.00207659]
Model: <catboost.core.CatBoostClassifier object at 0x792151d2bc40>, Proba: [0.0321734  0.08612591 0.11094129 ... 0.24001366 0.10362261 0.00467649]
Model: <catboost.core.CatBoostClassifier object at 0x79214ece05b0>, Proba: [0.04981967 0.1179583  0.1768586  ... 0.37035512 0.15525074 0.00634461]
Model: <catboost.core.CatBoostClassifier object at 0x79214d631750>, Proba: [0.06676685 0.15632629 0.24785876 ... 0.47941898 0.20764124 0.00980316]
Model: <catboost.core.CatBoostClassifier object at 0x792150d84310>, Proba: [0.07978033 0.18910931 0.29582475 ... 0.59581339 0.25325467 0.01128858]


In [55]:
count = 0

In [142]:

from datetime import datetime
import pytz
# カウント変数をインクリメント
count += 1
# 日本時間を取得
japan_tz = pytz.timezone('Asia/Tokyo')
now = datetime.now(japan_tz)
timestamp = now.strftime("%Y%m%d_%H%M%S")

file_name = f"/content/drive/MyDrive/Signate/2024summer/catboost_{timestamp}_{count:03d}.csv"
ss[1] = preds_y
ss.to_csv(file_name, header=False, index=False)